# Imports

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

import datetime

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Get Data

In [ ]:
print(datetime.datetime.now())

Base_Dados = pd.read_parquet('../data/base_consolidada/Base_Consolidada.parquet')

print(datetime.datetime.now())

# EDA

In [ ]:
type(Base_Dados)

In [ ]:
Base_Dados.columns

In [ ]:
Base_Dados.info()

In [ ]:
Base_Dados.isnull().sum()

In [ ]:
Base_Dados.nunique()

In [ ]:
Base_Dados.head()

In [ ]:
Total_multa = int(Base_Dados.shape[0])
print(f'Total de multas nos 12 meses: {Total_multa}')

In [ ]:
# Converter Data
Base_Dados['Data da Infração (DD/MM/AAAA)'] = pd.to_datetime(Base_Dados['Data da Infração (DD/MM/AAAA)'])

In [ ]:
# Multas por dia
Multas_Dia = Base_Dados['Data da Infração (DD/MM/AAAA)'].value_counts().sort_index().reset_index()

In [ ]:


# Extraíndo mês e dia das datas
Multas_Dia['Mes'] = pd.to_datetime(Multas_Dia['Data da Infração (DD/MM/AAAA)']).dt.month
Multas_Dia['Dia'] = pd.to_datetime(Multas_Dia['Data da Infração (DD/MM/AAAA)']).dt.month

# Renomeando colunas
Multas_Dia.columns = ['Data', 'Quantidade', 'Mes', 'Dia']

In [ ]:
Multas_Dia.head()

In [ ]:
Multas_Dia['Media_Movel'] = Multas_Dia['Quantidade'].rolling(7).mean()

In [ ]:
Multas_Dia.info()

# Análise Gráfica

In [ ]:
px.line(Multas_Dia, x = 'Data', y = 'Quantidade')

In [ ]:
go.Figure(go.Scatter(x = Multas_Dia['Data'], y = Multas_Dia['Quantidade']))

In [ ]:
# Sistema Grid

Grid = make_subplots(rows = 1, cols = 2)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Quantidade'],
    mode = 'lines', name = 'Quantidade'), row = 1, col = 1)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Quantidade'],
    mode = 'lines', name = 'Média Móvel'), row = 1, col = 2)

Grid.update_layout(
    title = 'Sistema de Grid',
    showlegend = True,
    legend = dict(
        orientation = 'h',
        yanchor = 'bottom',
        y = 1.02,
        xanchor = 'right',
        x = 1
    )
)

Grid


In [ ]:
# Sistema Grid

Grid = make_subplots(rows = 1, cols = 2)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Quantidade'],
    mode = 'lines', name = 'Quantidade'), row = 1, col = 1)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Media_Movel'],
    mode = 'lines', name = 'Média Móvel'), row = 1, col = 1)

Grid.update_layout(
    title = 'Análise de multas diária',
    showlegend = True,
    legend = dict(
        orientation = 'h',
        yanchor = 'bottom',
        y = 1.02,
        xanchor = 'right',
        x = 1
    )
)

Grid


In [ ]:
# Boxplot
px.box(Multas_Dia,
       x = 'Mes', 
       y = 'Quantidade',
       color = 'Mes',
       title = 'Distribuição de multas mês')

## Estados

In [ ]:
Acumulado_Estados = Base_Dados['UF Infração'].value_counts()
Acumulado_Estados_Perc = Base_Dados['UF Infração'].value_counts(normalize = True)
Acumulado_Estados_Perc_Acum = Base_Dados['UF Infração'].value_counts(normalize = True).cumsum()

# Dict
Dicionario = {
    'Estados': Acumulado_Estados.index,
    'Qtd Multas': Acumulado_Estados.values,
    'Representação': Acumulado_Estados_Perc.values,
    'Acumulado': Acumulado_Estados_Perc_Acum.values
}

Tabela_Estados = pd.DataFrame(Dicionario)

Tabela_Estados.head()

In [ ]:
px.funnel(
    Tabela_Estados[Tabela_Estados['Acumulado'] < 0.5],
    y = 'Estados',
    x = 'Qtd Multas',
    title = 'Concentração dos 50%'
)

In [ ]:
px.bar(
    Tabela_Estados,
    x = 'Estados',
    y = 'Acumulado',
    title = 'Multas Acumuladas por Estado'
)

## Heatmap

In [ ]:
Base_Dados['Mes'] = Base_Dados['Data da Infração (DD/MM/AAAA)'].dt.month

In [ ]:
Anl_Estado_Mes = Base_Dados.groupby(by = ['Mes', 'UF Infração']).agg(Quantidade = ('Município', 'count')).reset_index()

In [ ]:
Anl_Estado_Mes = Anl_Estado_Mes.pivot_table(index = 'Mes', columns = 'UF Infração', values = 'Quantidade')
Anl_Estado_Mes

In [ ]:
px.imshow(Anl_Estado_Mes, title = 'Mapa de calorMapa de calor | Multas mensais por estado em 2022')

In [ ]:
Base_Dados['Enquadramento da Infração'].value_counts(normalize = True).cumsum() * 100

Não há o valor das multas na base de dados, porém o enquadramento e no site do DETRAN, há essa informação. Vamos fazer Web Scrapping para conseguir estes dados.

# Web Scrapping

In [7]:
Url = 'https://www.detran.mg.gov.br/infracoes/consultar-tipos-infracoes/index/index/index/index/index/index/index/index/index/index/index/index/lista-de-infracoes?artigo=&descricao='

pd.read_html(Url)[0].head()

,Código,Desd.,Infração,Artigo,Infrator,Pts,Valor
0,5622,4,PARAR NOS CANTEIROS CENTRAIS OU DIVISORES DE P...,182 * VI,CON,3,8838
1,5622,5,PARAR NAS MARCAS DE CANALIZACAO,182 * VI,CON,3,8838
2,5630,0,PARAR NA AREA DE CRUZAMENTO DE VIAS,182 * VII,CON,4,13016
3,5649,1,PARAR NOS VIADUTOS,182 * VIII,CON,4,13016
4,5649,2,PARAR NAS PONTES,182 * VIII,CON,4,13016


In [ ]:
# Loop no site do Detran e buscar os dados
Url = 'https://www.detran.mg.gov.br/infracoes/consultar-tipos-infracoes/index/index/index/index/index/index/index/index/index/index/index/index/index/lista-de-infracoes?artigo=&descricao=&page='

# Tabela vazia
Base_Consolidada = pd.DataFrame()

for Loop in range(1, 24):
    
    # Construindo a URL
    Link = f'{Url}{Loop}'
    
    # Lendo os dados da Web
    Dados_web = pd.read_html(Link)[0]
    
    # Consolidar
    Base_Consolidada = pd.concat([Base_Consolidada, Dados_web])


In [ ]:
Base_Consolidada.shape

In [ ]:
Base_Consolidada.head()

In [ ]:
Base_Consolidada['Valor'] = pd.to_numeric(Base_Consolidada['Valor'], errors='coerce')
Base_Consolidada = Base_Consolidada.dropna(subset=['Valor'])


In [ ]:
Base_Consolidada['Valor'] = Base_Consolidada['Valor'] / 100

In [ ]:
Base_Consolidada.rename(columns = {'Código': 'Código da Infração'}, inplace = True)

In [ ]:
Base_Consolidada['Código da Infração'] = Base_Consolidada['Código da Infração'].astype('int64')

In [ ]:
Tab_Preco = Base_Consolidada.groupby(by=['Código da Infração'])['Valor'].apply(lambda x: np.mean(x.astype(float))).reset_index()

In [ ]:
# Reencresver a coluna de Infração
Base_Dados['Código da Infração'] = Base_Dados['Código da Infração'].apply( lambda Loop : Loop[0:4] )

In [ ]:
# Converto para numero o codigo
Base_Dados['Código da Infração'] = pd.to_numeric( Base_Dados['Código da Infração'], errors = 'coerce')

In [ ]:
Base_Dados['Código da Infração'].dtypes

In [ ]:
Tab_Preco['Código da Infração'].dtypes

In [ ]:
# Cruzando os dados
Cruzamento = pd.merge( Base_Dados, Tab_Preco, on='Código da Infração', how='left' )
Cruzamento.head()

In [ ]:
# Analise por UF e Preço
Tab_Soma = Cruzamento.groupby(by='UF Infração').agg(
    {'Valor':['count', 'sum']},
)

Tab_Soma.columns = Tab_Soma.columns.droplevel()

Tab_Soma = Tab_Soma.reset_index()

Tab_Soma.head()

In [ ]:
px.scatter( 
    Tab_Soma, 
    x='count', 
    y='sum', 
    color='UF Infração', 
    size='count', 
    log_x=True, 
    size_max=60, 
    title='Bubble PLOT' )

In [ ]:
fig = px.scatter( Tab_Soma, x='count', y='sum', color='UF Infração', title='Scatter PLOT')
fig.update_traces(marker=dict(size=12) )

In [ ]:
Analise_Valor_Mes = Cruzamento.groupby( by=['Mes'] )['Valor'].sum().reset_index()
Analise_Valor_Mes